In [2]:
import pandas as pd
from peewee import *

In [4]:
database = MySQLDatabase('wgs', **{'charset': 'utf8', 'sql_mode': 'PIPES_AS_CONCAT', 'use_unicode': True, 'host': 'localhost', 'port': 3306, 'user': 'root', 'password': '179353'})

class BaseModel(Model):
    class Meta:
        database = database

class QcBeforeFiltering(BaseModel):
    batch=CharField()
    Samples = CharField()
    Raw_total_reads=BigIntegerField()
    Raw_total_bases=BigIntegerField()
    Raw_q20_bases=BigIntegerField()
    Raw_q30_bases=BigIntegerField()
    Raw_q20_rate=FloatField()
    Raw_q30_rate=FloatField()
    Raw_read1_mean_length=IntegerField()
    Raw_read2_mean_length=IntegerField()
    Raw_gc_content=FloatField()

    class Meta:
        table_name = 'before_filtering'


class QcAfterFiltering(BaseModel):
    batch=CharField()
    Samples = CharField()
    Clean_total_reads=BigIntegerField()
    Clean_total_bases=BigIntegerField()
    Clean_q20_bases=BigIntegerField()
    Clean_q30_bases=BigIntegerField()
    Clean_q20_rate=FloatField()
    Clean_q30_rate=FloatField()
    Clean_read1_mean_length=IntegerField()
    Clean_read2_mean_length=IntegerField()
    Clean_gc_content=FloatField()

    class Meta:
        table_name = 'after_filtering'



class QcFilteringResult(BaseModel):
    batch=CharField()
    Samples = CharField()
    passed_filter_reads=BigIntegerField()
    low_quality_reads=BigIntegerField()
    too_many_N_reads=BigIntegerField()
    too_short_reads=BigIntegerField()
    too_long_reads=BigIntegerField()
    class Meta:
        table_name = 'filtering_result'
        
class Admixtools(BaseModel):
    batch = CharField()
    A = CharField()
    B = CharField()
    X = CharField()
    C = CharField()
    O = CharField()
    alpha = FloatField()
    stderr = FloatField()
    Zscore = FloatField()

    class Meta:
        table_name = 'qpf4ratio'

def create_table(table):
    u"""
    如果table不存在，新建table
    """
    if not table.table_exists():
        table.create_table()

def drop_table(table):
    u"""
    table 存在，就删除
    """
    if table.table_exists():
        table.drop_table()
        
class SampleInfo(BaseModel):
    batch = CharField()
    fq1 = CharField()
    fq2 = CharField()
    adapter = CharField()
    library = CharField()
    sample = CharField()
    class Meta:
        table_name = 'sample_info'
        
class KilogenomePop(BaseModel):
    continent = CharField()
    country = CharField()
    population = CharField()

    class Meta:
        table_name = 'kilogenome_pop'
        
        
class Admixture(BaseModel):
    batch = CharField()
    Samples = CharField()
    GBR = FloatField()
    FIN = FloatField()
    CHS = FloatField()
    PUR = FloatField()
    CDX = FloatField()
    CLM = FloatField()
    IBS = FloatField()
    PEL = FloatField()
    PJL = FloatField()
    KHV = FloatField()
    ACB = FloatField()
    GWD = FloatField()
    ESN = FloatField()
    BEB = FloatField()
    MSL = FloatField()
    STU = FloatField()
    ITU = FloatField()
    CEU = FloatField()
    YRI = FloatField()
    CHB = FloatField()
    JPT = FloatField()
    LWK = FloatField()
    ASW = FloatField()
    MXL = FloatField()
    TSI = FloatField()
    GIH = FloatField()

    class Meta:
        table_name = 'Admixture_stat'

In [306]:

create_table(SampleInfo)


In [121]:
drop_table(QcBeforeFiltering)
drop_table(QcAfterFiltering)
drop_table(QcFilterResult)

In [301]:




database = MySQLDatabase('wgs', **{'charset': 'utf8', 'sql_mode': 'PIPES_AS_CONCAT', 'use_unicode': True, 'host': 'localhost', 'port': 3306, 'user': 'root', 'password': '179353'})



class Base(object):


    def __init__(self,batch):
        self.batch=batch
        self.context=self.batch.split('_')
        self.username=self.context[-1]
        self.analysis=self.context[0]
        self.project='_'.join(self.context[1:-1])
        self.save_dir=os.path.join('/Users/linlian/Documents/GitHub/Keygene/Keygene/media/file',self.username)
        self.save_dir=os.path.join(self.save_dir,self.project)
        self.save_dir=os.path.join(self.save_dir,self.analysis)



    def create_json_list(self,df,table):
        df['batch']=self.batch
        test=json.loads(df.to_json(orient='records'))
        create_table(table)
        with database.atomic():
            for i in range(0, len(test), 100):
                table.insert_many(test[i:i+100]).execute()



class Qc(Base):
    def __init__(self,batch):
        super(Qc,self).__init__(batch)
        self.after_filtering_sql()
        self.before_filtering_sql()
        self.filtering_result_sql()


    def after_filtering_sql(self):
        df=pd.read_csv(os.path.join(self.save_dir,'qc/after_filtering.csv'))
        self.create_json_list(df,QcAfterFiltering)
        

    def before_filtering_sql(self):
        df=pd.read_csv(os.path.join(self.save_dir,'qc/before_filtering.csv'))
        self.create_json_list(df,QcBeforeFiltering)

    def filtering_result_sql(self):
        df=pd.read_csv(os.path.join(self.save_dir,'qc/filtering_result.csv'))
        self.create_json_list(df,QcFilteringResult)
        
        
class InputInfo(Base):
    def __init__(self,batch):
        super(InputInfo,self).__init__(batch)

    def info(self):
        data=[]
        with open(os.path.join(self.save_dir,'input.txt'),'rt') as f:
            for k,v in enumerate(f):
                if k == 0:
                    pass
                else:
                    line=v.split()
                    fq1=line[0]
                    fq2=line[1]
                    adapter=line[2]
                    library=line[3]
                    sample=line[4]
                    data.append({'batch':self.batch,'fq1':fq1,'fq2':fq2,'adapter':adapter,'library':library,'sample':sample})
        self.insert_db(SampleInfo,data)


In [27]:
kg = {'东亚': ['CHB', 'JPT', 'CHS', 'KHV', 'CDX'], '南亚': ['ITU', 'BEB', 'STU', 'GIH', 'PJL'], '欧洲': [
        'FIN', 'CEU', 'TSI', 'GBR', 'IBS'], '美洲': ['PEL', 'MXL', 'CLM', 'PUR'], 
        '非洲': ['ASW', 'ACB', 'GWD', 'ESN', 'LWK', 'MSL', 'YRI']}

In [5]:
batch='demo_demo_lianlin_lianlin'

In [13]:
table=Admixture.select().where(Admixture.batch==batch)
kg=KilogenomePop.select()

In [14]:
df=pd.DataFrame.from_dict(table.dicts())

In [18]:
kilogenome=pd.DataFrame.from_dict(kg.dicts())

In [19]:
population_country=dict(zip(kilogenome['population'],kilogenome['country']))

In [22]:
df2=df[df['Samples'] == 'P002190812016']
df2.set_index('Samples',inplace=True)

In [24]:
pop=[]
for p in kg:
    eas=df2[kg[p]]
    eas=eas.reset_index()
    eas_sub=defaultdict(list)
    eas_sub=df_tuple(eas_sub,eas)
    eas_sub=eas_sub[sample]
    eas_sub=sorted(eas_sub,key=lambda x:x[1],reverse=True)
    eas_sub=list(map(lambda x:[x[0],format(x[1], '.4%')],eas_sub))
    pop.append((p,eas_sub))

TypeError: '>=' not supported between instances of 'KilogenomePop' and 'int'

In [28]:
df2[kg['东亚']]

,CHB,JPT,CHS,KHV,CDX
Samples,,,,,
P002190812016,0.495569,0.052568,0.400288,0.00001,0.051354


In [30]:
kg['东亚']

['CHB', 'JPT', 'CHS', 'KHV', 'CDX']

In [29]:
kg

{'东亚': ['CHB', 'JPT', 'CHS', 'KHV', 'CDX'],
 '南亚': ['ITU', 'BEB', 'STU', 'GIH', 'PJL'],
 '欧洲': ['FIN', 'CEU', 'TSI', 'GBR', 'IBS'],
 '美洲': ['PEL', 'MXL', 'CLM', 'PUR'],
 '非洲': ['ASW', 'ACB', 'GWD', 'ESN', 'LWK', 'MSL', 'YRI']}